In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
os.listdir("../input")

In [ ]:
#libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error

import matplotlib.pyplot as plt
import seaborn as sns

from lightgbm import LGBMRegressor as lgbm

Encode = OneHotEncoder(handle_unknown = 'ignore', sparse = False)
#from mpl_toolkits.mplot3d import Axes3D

In [ ]:
t = pd.DataFrame({'A':[1,2,3],'B':[4,5,6],'C':[7,8,9]})
t.apply(np.sum,axis=0)

In [ ]:
"""
User defined functions
"""
#This function returns z-scaled values along with mean and std
def normalise(df):
    return (df - np.mean(df))/np.mean(df), np.mean(df), np.std(df)

#This function returns electronegativity values
def nega(df):
    nega_dict = {'C':2.55, 'H':2.2, 'N':3.04, 'O':3.44, 'F':3.98}
    return [nega_dict[var] for var in df]

#This function returns covalent radius for each atom
def radii(df):
    radii_dict = {'C':73, 'H':31, 'N':71, 'O':66, 'F':57}
    return [radii_dict[var] for var in df]

#This function returns the bondlength distance
def distance (df):
    return np.sqrt((df.x1 - df.x0)**2 + (df.y1 - df.y0)**2 + (df.z1 - df.z0)**2)

# def EN_vectors(df):
def norm_ENV (length, coord, EN):
    return  EN*coord/length

def checkdir (x0,y0,z0,x1,y1,z1):
    diff = np.sqrt((x1 - x0)**2 + (y1 - y0)**2 + (z1 - z0)**2)
    summ = np.sqrt((x1 + x0)**2 + (y1 + y0)**2 + (z1 + z0)**2)
    if summ >= diff:
        return 1
    else:
        return -1
    #if len(sum) > len
    
# def ENV_res(row):
#     temp = train_X[['V_x','V_y','V_z']].loc[(molecule_name == row.molecule_name) & (atom_index0 == row.atom_index0)].apply(np.sum,axis =0)
#     return temp

def ENV_res_x(row):
    temp = train_X.loc[(train_X.molecule_name == row[0]) & (train_X.atom_index0 == row[2])]
    return np.sum(temp.V_x)

def ENV_res_y(row):
    temp = train_X.loc[(train_X.molecule_name == row.molecule_name) & (train_X.atom_index0 == row.atom_index0)]
    return np.sum(temp.V_y)

def ENV_res_z(row):
    temp = train_X.loc[(train_X.molecule_name == row.molecule_name) & (train_X.atom_index0 == row.atom_index0)]
    return np.sum(temp.V_z)

In [ ]:
trial_test = pd.DataFrame()
trial_test.assign([x,y,z]= [1,2,4])

In [ ]:
""" LOG MAE score calculator
    taken from the public kernel - https://www.kaggle.com/marcelotamashiro/lgb-public-kernels-plus-more-features @marcelotomashiro
"""

def comp_score (y_true, y_pred, jtype):
    df = pd.DataFrame()
    df['y_true'] , df['y_pred'], df['jtype'] = y_true , y_pred, jtype
    score = 0 
    for t in jtype.unique():
        score_jtype = np.log(mean_absolute_error(df[df.jtype==t]['y_true'],df[df.jtype==t]['y_pred']))
        score += score_jtype
        print(f'{t} : {score_jtype}')
    score /= len(jtype.unique())
    return score

In [ ]:
train = pd.read_csv("../input/champs-scalar-coupling/train.csv")
structure = pd.read_csv("../input/champs-scalar-coupling/structures.csv")

train_X = train[train.columns[1:-1]]
train_y = train[train.columns[-1]]

train_X = pd.merge(train_X,structure, left_on=['molecule_name','atom_index_0'], right_on=['molecule_name','atom_index'], how='left',suffixes =('0','1'))
train_X = train_X.assign(EN0 = nega(train_X.atom))
train_X = pd.merge(train_X,structure, left_on=['molecule_name','atom_index_1'], right_on=['molecule_name','atom_index'], how='left',suffixes =('0','1'))
train_X = train_X.assign(EN1 = nega(train_X.atom1))
train_X = train_X.assign(REN = train_X.EN1 - train_X.EN0)
train_X = train_X.drop(['atom_index_1','atom_index_0'], axis =1)

# distance_norm, distance_mean, distance_std = normalise(distance(train_X))


# train_X = train_X.assign(x1x0 = train_X.x1 - train_X.x0)
# train_X = train_X.assign(y1y0 = train_X.y1 - train_X.y0)
# train_X = train_X.assign(z1z0 = train_X.z1 - train_X.z0)

distance_V = distance(train_X)
train_X = train_X.assign(V_x = norm_ENV(distance_V, train_X.x1 - train_X.x0, (train_X.EN1 - train_X.EN0)/distance_V))
train_X = train_X.assign(V_y = norm_ENV(distance_V, train_X.y1 - train_X.y0, (train_X.EN1 - train_X.EN0)/distance_V))
train_X = train_X.assign(V_z = norm_ENV(distance_V, train_X.z1 - train_X.z0, (train_X.EN1 - train_X.EN0)/distance_V))

#TEST DATA
test = pd.read_csv("../input/champs-scalar-coupling/test.csv")
test_X = test

test_X = pd.merge(test_X, structure, left_on=['molecule_name','atom_index_0'], right_on=['molecule_name','atom_index'], how='left',suffixes =('0','1'))
test_X = test_X.assign(EN0 = nega(test_X.atom))
test_X = pd.merge(test_X, structure, left_on=['molecule_name','atom_index_1'], right_on=['molecule_name','atom_index'], how='left',suffixes =('0','1'))
test_X = test_X.assign(EN1 = nega(test_X.atom1))
test_X = test_X.assign(REN = train_X.EN1 - train_X.EN0)
test_X = test_X.drop(['atom_index_1','atom_index_0'], axis =1)

distance_y = distance(test_X)
test_X = test_X.assign(V_x = norm_ENV(distance_y, test_X.x1 - test_X.x0, (test_X.EN1 - test_X.EN0)/distance_y))
test_X = test_X.assign(V_y = norm_ENV(distance_y, test_X.z1 - test_X.y0, (test_X.EN1 - test_X.EN0)/distance_y))
test_X = test_X.assign(V_z = norm_ENV(distance_y, test_X.z1 - test_X.z0, (test_X.EN1 - test_X.EN0)/distance_y))


train_X.head(10)

In [ ]:
test_X.head(5)

In [ ]:
# train_X = train_X.assign(ENV_x = train_X.values.apply(ENV_res_x, axis =1))
# train_X = train_X.assign(ENV_y = train_X.values.apply(ENV_res_y, axis =1))
# train_X = train_X.assign(ENV_z = train_X.values.apply(ENV_res_z, axis =1))

train_X = train_X.assign(ENV_x =[ENV_res_x(x) for x in train_X.values])

In [ ]:
# train_X.loc[(train_X.molecule_name == train_X.molecule_name.unique()[0])]
# train_X.loc[((train_X.atom_index0 == 4))]
# train_X.loc[(train_X.molecule_name == train_X.molecule_name.unique()[0]) & (train_X.atom_index0 == 3)]
# train_X.loc[(train_X.molecule_name == train_X.molecule_name.unique()[0]) & ((train_X.atom_index0 == 3) | (train_X.atom_index1 == 3))]
train_X.loc[(train_X.molecule_name == train_X.molecule_name.unique()[7]) & ((train_X.atom0 == 'C') | (train_X.atom1 == 'C'))]

In [ ]:
sns.violinplot(train.type,train.scalar_coupling_constant)

In [ ]:
#training set
X = pd.DataFrame()
#assigning z scale of bondlength
CL, CL_mean, CL_std = normalise(distance(train_X))
X = X.assign(coupling_length = CL)


EN1, EN_mean1, EN_std1 = normalise(nega(train_X.atom1))
X = X.assign(E_neg1 = EN1)

radi1, radi_mean1, radi_std1 = normalise(radii(train_X.atom1))
X = X.assign(a_radius1 = radi1)

X = X.assign(norm_EN1 = train_X.REN/distance_V) #same as magnitude of the ENV vector

X = X.assign(V_x = train_X.V_x)
X = X.assign(V_y = train_X.V_y)
X = X.assign(V_z = train_X.V_z)

# X = X.assign(ENV1_mag = np.sqrt(train_X.V_x**2 + train_X.V_y**2 + train_X.V_z**2)) #same as norm_EN1

type_col  = pd.DataFrame(Encode.fit_transform(np.array(train_X['type']).reshape(-1,1)))

X = pd.concat([X, type_col], axis=1)

X.head(10)

In [ ]:
X_1JHC = X.loc[train_X.type == '1JHC'] 
X_2JHH = X.loc[train_X.type == '2JHH']
X_1JHN = X.loc[train_X.type == '1JHN']
X_rest = X.loc[(train_X.type != '1JHC') & (train_X.type != '2JHH') & (train_X.type != '1JHN')]

y_1JHC = train_y.loc[train_X.type == '1JHC']
y_2JHH = train_y.loc[train_X.type == '2JHH']
y_1JHN = train_y.loc[train_X.type == '1JHN']
y_rest = train_y.loc[(train_X.type != '1JHC') & (train_X.type != '2JHH') & (train_X.type != '1JHN')]

norm_y_1JHC, y_1JHC_mean, y_1JHC_std = normalise(y_1JHC)
norm_y_2JHH, y_2JHH_mean, y_2JHH_std = normalise(y_2JHH)
norm_y_1JHN, y_1JHN_mean, y_1JHN_Std = normalise(y_1JHN)
norm_y_rest, y_rest_mean, y_rest_std = normalise(y_rest)




In [ ]:
y_rest_mean

In [ ]:
train_X.type.value_counts()

In [ ]:
X_t_1JHC, X_v_1JHC, y_t_1JHC, y_v_1JHC = train_test_split(X_1JHC,norm_y_1JHC, test_size=0.3, random_state= 0)

In [ ]:
"""
from lightgbm import LGBMRegressor
my_lgbm_model = LGBMRegressor(max_depth = -1, n_estimators =400, learning_rate =0.01, random_state =0)
my_lgbm_model.fit(X_train, y_train, eval_metric ='mae')
val_predictions_lgbm = my_lgbm_model.predict(X_valid)

"""
from lightgbm import LGBMRegressor as lgbm

lgbm_1JHC = lgbm(max_depth = -1, n_estimators =5000, learning_rate =0.1, random_state =0)
lgbm_1JHC.fit(X_t_1JHC, y_t_1JHC, eval_metric ='mae')

predictions_1JHC = lgbm_1JHC.predict(X_v_1JHC)

mean_absolute_error(predictions_1JHC, y_v_1JHC)


In [ ]:
X_t_rest, X_v_rest, y_t_rest, y_v_rest = train_test_split(X_rest,norm_y_rest, test_size=0.3, random_state= 0)
lgbm_rest = lgbm(max_depth = -1, learning_rate = 0.1, random_state =0)
lgbm_rest.fit(X_t_rest, y_t_rest, eval_metric ='mae')

predictions_rest = lgbm_rest.predict(X_v_rest)

mean_absolute_error(predictions_rest, y_v_rest)
